## [네이버 영화 상영작 정보 수집하기](https://movie.naver.com/movie/running/current.naver)
### 2022.01.28

In [49]:
# 수집한 데이터 저장용
import numpy as np
import pandas as pd
# 요청
import requests
# 수집
import bs4
# 기타
import time
import os
from IPython.display import clear_output

In [2]:
# 요청함수
def getSource(site) :
    # 헤더 정보 셋팅
    header_info = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
    }
    
    # 요청한다
    response = requests.get(site, headers=header_info)
    return response.text.strip()

In [181]:
# 한 페이지의 데이터를 수집해 저장하는 함수
def getData(source, fileName) :
    # 데이터를 담을 딕셔너리
    data_dict = {
        '영화제목' : [],
        '예매율' : [],
        '개요' : [], 
        '감독' : [],
        '배우' : [],
        '개봉일' : [],
        '상영시간' : [],
        '평점' : [],
        '영화코드' : []
    }
    
    # 영화 목록(li 태그)들을 가져온다.
    li_tags = source.select('#content > div.article > div:nth-child(1) > div.lst_wrap > ul > li')
    
    for li in li_tags :
#         print(li)
#         print('-----------------------')
        
        # 영화 제목, 영화 코드
        a1 = li.select_one('dl > dt > a')
        
        if a1 == None :
            data_dict['영화제목'].append(np.nan)
            data_dict['영화코드'].append(np.nan)
        else :
            data1 = a1.text.strip()
            data_dict['영화제목'].append(data1)
            # 링크 주소를 가져온다.
            href = a1.attrs['href']
            a3 = href.split('=')
            data_dict['영화코드'].append(a3[1])
        
        # 예매율
        a1 = li.select_one('dl > dd.star > dl.info_exp > dd > div > span.num')

        if a1 == None :
            data_dict['예매율'].append(np.nan)
        else :
            data1 = a1.text.strip()
            data_dict['예매율'].append(data1)
        
        # 개요
        a1 = li.select_one('dl > dd:nth-child(3) > dl > dd:nth-child(2) > span.link_txt')

        if a1 == None :
            data_dict['개요'].append(np.nan)
            
        else:
            data1 = a1.text.strip()
            data1 = removeChar(data1)
            data_dict['개요'].append(data1)
        
        # 감독
        a1 = li.select_one('dl > dd:nth-child(3) > dl > dd:nth-child(4) > span')

        if a1 == None :
            data_dict['감독'].append(np.nan)
        else :
            data1 = a1.text.strip()
            data1 = removeChar(data1)
            data_dict['감독'].append(data1)
        
        # 배우
        a1 = li.select_one('dl > dd:nth-child(3) > dl > dd:nth-child(6) > span')
        
        if a1 == None :
            data_dict['배우'].append(np.nan)
        else :
            data1 = a1.text.strip()
            data1 = removeChar(data1)
            data_dict['배우'].append(data1)
        
        # 개봉일, 상영시간
        a1 = li.select_one('dl > dd:nth-child(3) > dl > dd:nth-child(2)')
        
        if a1 == None :
            data_dict['상영시간'].append(np.nan)
            data_dict['개봉일'].append(np.nan)
        else :
            data1 = a1.text.strip()
            data1 = removeChar(data1)
            # | 를 기준으로 잘라낸다.
            data2 = data1.split('|')
            data_dict['상영시간'].append(data2[1][:-1])
            data_dict['개봉일'].append(data2[2][:-3])
        
        # 평점
        a1 = li.select_one('dl > dd.star > dl.info_star > dd > div > a > span.num')

        if a1 == None :
            data_dict['평점'].append(np.nan)
        else :
            data1 = a1.text.strip()
            data_dict['평점'].append(data1)
    

#     print(len(data_dict['영화제목']))    
#     print(len(data_dict['예매율']))
#     print(len(data_dict['개요']))
#     print(len(data_dict['감독']))
#     print(len(data_dict['배우']))
#     print(len(data_dict['개봉일']))
#     print(len(data_dict['상영시간']))
#     print(len(data_dict['평점']))
#     print(len(data_dict['영화코드']))


#     # 데이터 프레임을 생성한다.
    df1 = pd.DataFrame(data_dict)
#     display(df1)
    
    # 저장한다.
    # 파일이 없다면 : 첫 번째 저장
    if os.path.exists(fileName) == False :
        df1.to_csv(fileName, encoding='utf-8-sig', index=False)
    # 파일이 있다면..
    else :
        df1.to_csv(fileName, encoding='utf-8-sig', index=False, mode='a', header=None)

In [182]:
# 다음 페이지 주소를 가져오는 함수
def getNextPage(source) :
    pass

In [183]:
# 파일명 정제
def fileNameCheck(fileName) :
    # 윈도우에서 허용하지 않는 문자는 모두 제외한다.
    fileName = fileName.replace('\\', ' ')
    fileName = fileName.replace('/', ' ')
    fileName = fileName.replace(':', ' ')
    fileName = fileName.replace('*', ' ')
    fileName = fileName.replace('?', ' ')
    fileName = fileName.replace('"', ' ')
    fileName = fileName.replace('<', ' ')
    fileName = fileName.replace('>', ' ')
    fileName = fileName.replace('|', ' ')
    
    return fileName

In [184]:
# 불필요한 문자를 제거한다.
def removeChar(data) :
    data = data.replace('\r', '')
    data = data.replace('\n', '')
    data = data.replace('\t', '')
    data = data.replace(', ', '_')
    
    return data

In [185]:
# 페이지의 주소
site = 'https://movie.naver.com/movie/running/current.naver'
nextPage = ''
fileName = 'naver_movie_info.csv'

# 파일이 존재하면 삭제한다.
if os.path.exists(fileName) == True :
    os.remove(fileName)
    
# 서버에 접속해서 응답 결과를 가져온다.
a1 = getSource(f'{site}{nextPage}')
source = bs4.BeautifulSoup(a1)

# 현재 페이지의 데이터를 수집한다.
getData(source, fileName)

print('수집 완료')

수집 완료
